In [1]:
from ortools.linear_solver import pywraplp

In [2]:
# Define available resources
resources = ['A1', 'A2', 'B1', 'B2', 'C1', 'C2']

# Define the intents.
# For have, the first value is the amount to trade and the second is how much the intent holder values that resource.
# These are interpreted as exclusive possible trades; one thing gets traded for one other thing, and only one of the two options.
intents = [
    #intentA
    {'weight': 1.0, 'have': {'A1': (1, 1.0), 'A2': (1, 0.5)}, 
                    'want': {'C1': (1, 1.0), 'B2': (1, 1.0)}},
    #intentB
    {'weight': 1.0, 'have': {'B1': (1, 0.5), 'B2': (1, 1.0)}, 
                    'want': {'C2': (1, 1.0), 'A2': (1, 0.2)}},
    #intentC
    {'weight': 1.0, 'have': {'C1': (1, 1.0), 'C2': (1, 1.0)}, 
                    'want': {'B1': (1, 1.0), 'A1': (1, 1.0)}},
]

In [3]:
# Initialize the solver
solver = pywraplp.Solver.CreateSolver('SCIP')

# Objective function
objective = solver.Objective()

# Create decision variables
have_vars = {}
want_vars = {}

for i, intent in enumerate(intents):
    for have in intent['have']:
        have_vars[f"h_{i}_{have}"] = solver.BoolVar(f"h_{i}_{have}")
    for want in intent['want']:
        want_vars[f"w_{i}_{want}"] = solver.BoolVar(f"w_{i}_{want}")

# Add constraints that only one 'have' and one 'want' can be selected for each intent
for i, intent in enumerate(intents):
    have_sum = sum(have_vars[f"h_{i}_{have}"] for have in intent['have'])
    want_sum = sum(want_vars[f"w_{i}_{want}"] for want in intent['want'])
    solver.Add(have_sum <= 1)
    solver.Add(want_sum <= 1)
    solver.Add(have_sum == want_sum)

# Conservation laws for each resource
for resource in resources:
    inflow = sum(qty * have_vars[f"h_{i}_{have}"] for i, intent in enumerate(intents) for have, (qty, _) in intent['have'].items() if have == resource)
    outflow = sum(qty * want_vars[f"w_{i}_{want}"] for i, intent in enumerate(intents) for want, (qty, _) in intent['want'].items() if want == resource)
    solver.Add(inflow == outflow)
    
# Objective function
objective = solver.Objective()
for i, intent in enumerate(intents):
    for have, (_, have_weight) in intent['have'].items():
        for want, (_, want_weight) in intent['want'].items():
            have_coefficient = intent['weight'] * (1/have_weight)
            objective.SetCoefficient(have_vars[f"h_{i}_{have}"], have_coefficient)
            want_coefficient = intent['weight'] * want_weight
            objective.SetCoefficient(want_vars[f"w_{i}_{want}"], want_coefficient)


# Set objective function to maximize
objective.SetMaximization()

In [4]:
# Solve the problem
status = solver.Solve()

In [5]:
# Output results
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f"Objective value = {solver.Objective().Value()}")
    
    for i, intent in enumerate(intents):
        for have in intent['have']:
            if have_vars[f"h_{i}_{have}"].solution_value() > 0:
                print(f"Intent {i+1} gives {have}")
        
        for want in intent['want']:
            if want_vars[f"w_{i}_{want}"].solution_value() > 0:
                print(f"Intent {i+1} receives {want}")
else:
    print("The problem does not have a solution.")

Objective value = 7.2
Intent 1 gives A2
Intent 1 receives C1
Intent 2 gives B1
Intent 2 receives A2
Intent 3 gives C1
Intent 3 receives B1
